In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('map_test').getOrCreate()

In [2]:
df = spark.read.csv('/FileStore/tables/convStore_done.csv', inferSchema=True, header=True)

In [4]:
from math import cos
def state_code_compute(x):    
    scale_km = 0.5
    lat = x[0]
    lon = x[1]
    lat_to_km = lat*110.574
    lon_to_km = abs(lon*(111.320*cos(lat)))

    state_code_x = str(int(lat_to_km/scale_km))
    state_code_y = str(int(lon_to_km/scale_km))
    if len(state_code_y) == 4:
        state_code_y = '0'+state_code_y
    state_code = state_code_x+state_code_y
    return state_code

In [5]:
f = df.select(['lat','lon']).rdd

In [7]:
f_t = f.map(state_code_compute)

In [9]:
from pyspark.sql import Row
f_ret = f_t.map(lambda x: Row(state_code=x))

In [10]:
state_code = spark.createDataFrame(f_ret)

In [11]:
from pyspark.sql.functions import monotonically_increasing_id
df = df.withColumn("id", monotonically_increasing_id())
state_code = state_code.withColumn("id", monotonically_increasing_id())
t = df.join(state_code, "id", "outer").drop("id")

In [13]:
t.write.csv('mycsv_.csv', header=True)

In [14]:
iii = spark.read.csv('mycsv_.csv', inferSchema=True, header=True)